In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import MEstimateEncoder, TargetEncoder
from sklearn.inspection import permutation_importance

import seaborn as sns
import shap

import lightgbm as lgb

import sys
sys.path.append('../lightgbm')

In [14]:
# File Paths
test_df_path = '../../Dataset/test.csv/test.csv'
process_train_path = './processed/processed_train_2.csv'
processed_test_path = './processed/processed_test_2.csv'
feats_path = './best_feats/Onehot_only_mean_NANs_modified_prices.csv'
output_path = './output/lgbm_pred_topFeat.csv'

top_n = 75

In [15]:
feats_df = pd.read_csv(feats_path)
feats = feats_df['col_name'].values.tolist()[:top_n]
processed_df = pd.read_csv(process_train_path)
processed_df = processed_df[feats+['price_doc']]
X = processed_df.drop(['price_doc'], axis=1)
y = processed_df['price_doc']
X = X[feats]

test_df = pd.read_csv(test_df_path)
processed_test_df = pd.read_csv(processed_test_path)
processed_test_df

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,...,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan
0,30474,39.00,20.700000,2,9,1998.000000,1,8.9,3.000000,2.615514e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30475,79.20,34.404467,8,17,0.000000,3,1.0,1.000000,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30476,40.50,25.100000,3,5,1960.000000,2,4.8,2.000000,9.946335e+06,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,30477,62.80,36.000000,17,17,2016.000000,2,62.8,3.000000,2.149409e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30478,40.00,40.000000,17,17,0.000000,1,1.0,1.000000,2.553630e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,38131,52.20,31.800000,10,12,1973.000000,2,9.1,2.000000,7.811375e+06,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7658,38132,54.09,34.404467,14,0,1879.046638,2,0.0,2.105145,5.299528e+07,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7659,38133,41.08,1.000000,12,1,1.000000,1,1.0,1.000000,7.307411e+06,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7660,38134,34.80,19.800000,8,9,1977.000000,1,6.4,2.000000,7.128794e+06,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.metrics import mean_squared_error
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "n_estimators": trial.suggest_int("n_estimators", 800, 1200),
#         "boosting": trial.suggest_categorical("boosting", ["gbdt", "rf", "dart"]),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 10),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0, 1),
        "bagging_freq": 1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2023-10-30 21:40:07,450] A new study created in memory with name: no-name-bfc67154-3467-46cf-972b-a7bfa4ae289b
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.6070130407242049, colsample_bytree=0.9356367928201346 will be ignored. Current value: feature_fraction=0.6070130407242049
[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=49
[LightGBM] [Warning] bagging_fraction is set=0.6804008636370036, subsample=0.433891617622374 will be ignored. Current value: bagging_fraction=0.6804008636370036
[LightGBM] [Warning] lambda_l2 is set=8.070371159313087, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.070371159313087
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 21:40:10,627] Trial 0 finished with value: 2618056.877218267 and parameters: {'n_estimators': 837, 'lambda_l2': 8.070371159313087, 'bagging_fraction': 0.6804008636370036, 'num_leaves': 973, 'feature_fraction': 0.6070130407242049, 'max_depth': 8, 'learning_rate': 0.004163955700652882, 'subsample': 0.433891617622374, 'colsample_bytree': 0.9356367928201346, 'min_data_in_leaf': 49}. Best is trial 0 with value: 2618056.877218267.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.820364070733431, colsample_bytree=0.4515205695085263 will be ignored. Current value: feature_fraction=0.820364070733431
[LightGBM] [Warning] min_data_in_leaf is set=34, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=34
[LightGBM] [Warning] bagging_fraction is set=0.09475506939480982, subsample=0.4902873656989232 will be ignored. Current value: bagging_fraction=0.09475506939480982
[LightGBM] [Warning] lambda_l2 is set=0.9026628948332671, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9026628948332671
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 21:40:14,082] Trial 1 finished with value: 2812628.002089625 and parameters: {'n_estimators': 999, 'lambda_l2': 0.9026628948332671, 'bagging_fraction': 0.09475506939480982, 'num_leaves': 812, 'feature_fraction': 0.820364070733431, 'max_depth': 43, 'learning_rate': 0.09692546105835612, 'subsample': 0.4902873656989232, 'colsample_bytree': 0.4515205695085263, 'min_data_in_leaf': 34}. Best is trial 0 with value: 2618056.877218267.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


...

[I 2023-10-30 21:45:24,358] Trial 47 finished with value: 2569331.270529251 and parameters: {'n_estimators': 949, 'lambda_l2': 0.41556391368440204, 'bagging_fraction': 0.8289522632264283, 'num_leaves': 190, 'feature_fraction': 0.7687793905867762, 'max_depth': 13, 'learning_rate': 0.004236763872087451, 'subsample': 0.8848217818822173, 'colsample_bytree': 0.622045909678827, 'min_data_in_leaf': 67}. Best is trial 9 with value: 2478494.487583945.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.7384381842733188, colsample_bytree=0.6692301024811949 will be ignored. Current value: feature_fraction=0.7384381842733188
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] bagging_fraction is set=0.9444037959357493, subsample=0.7469741206698409 will be ignored. Current value: bagging_fraction=0.9444037959357493
[LightGBM] [Warning] lambda_l2 is set=1.566186367849204, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.566186367849204
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 21:45:33,681] Trial 48 finished with value: 2492613.0572465486 and parameters: {'n_estimators': 883, 'lambda_l2': 1.566186367849204, 'bagging_fraction': 0.9444037959357493, 'num_leaves': 303, 'feature_fraction': 0.7384381842733188, 'max_depth': 11, 'learning_rate': 0.007477748918775924, 'subsample': 0.7469741206698409, 'colsample_bytree': 0.6692301024811949, 'min_data_in_leaf': 13}. Best is trial 9 with value: 2478494.487583945.
'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.


[LightGBM] [Warning] feature_fraction is set=0.831591996144048, colsample_bytree=0.4996688377466395 will be ignored. Current value: feature_fraction=0.831591996144048
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] bagging_fraction is set=0.8840030133934397, subsample=0.9298200607749817 will be ignored. Current value: bagging_fraction=0.8840030133934397
[LightGBM] [Warning] lambda_l2 is set=0.8909567854657356, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8909567854657356
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-10-30 21:45:42,521] Trial 49 finished with value: 2534601.5155673283 and parameters: {'n_estimators': 1002, 'lambda_l2': 0.8909567854657356, 'bagging_fraction': 0.8840030133934397, 'num_leaves': 543, 'feature_fraction': 0.831591996144048, 'max_depth': 24, 'learning_rate': 0.010961950013496138, 'subsample': 0.9298200607749817, 'colsample_bytree': 0.4996688377466395, 'min_data_in_leaf': 83}. Best is trial 9 with value: 2478494.487583945.


In [18]:
params = study.best_params
params["objective"] = "regression"
params["metric"] = "rmse"
params["verbosity"] = -1

#Best
params2 = {
    'boosting': 'gbdt', 
    'lambda_l2': 7.068023653932577, 
    'bagging_fraction': 0.6261319161311689, 
    'num_leaves': 151, 
    'feature_fraction': 0.976804739696195, 
    'max_depth': 8, 'learning_rate': 0.01552248392804131, 
    'subsample': 0.4408875142865268, 
    'colsample_bytree': 0.3829262582959811, 
    'min_data_in_leaf': 21,
    "objective": "regression",
    "metric": "rmse",
    "n_estimators": 1000,
    "verbosity": -1,
}

In [19]:
model = lgb.LGBMRegressor(**params2)

model.fit(X, y)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.976804739696195, colsample_bytree=0.3829262582959811 will be ignored. Current value: feature_fraction=0.976804739696195
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] bagging_fraction is set=0.6261319161311689, subsample=0.4408875142865268 will be ignored. Current value: bagging_fraction=0.6261319161311689
[LightGBM] [Warning] lambda_l2 is set=7.068023653932577, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.068023653932577


LGBMRegressor(bagging_fraction=0.6261319161311689, boosting='gbdt',
              colsample_bytree=0.3829262582959811,
              feature_fraction=0.976804739696195, lambda_l2=7.068023653932577,
              learning_rate=0.01552248392804131, max_depth=8, metric='rmse',
              min_data_in_leaf=21, n_estimators=1000, num_leaves=151,
              objective='regression', subsample=0.4408875142865268,
              verbosity=-1)

In [20]:
processed_test_df_copy = processed_test_df.copy()

processed_test_df = processed_test_df.drop(['id'], axis=1)

In [21]:
pred = model.predict(processed_test_df[feats])
prediction_df = pd.DataFrame({
    'id': processed_test_df_copy['id'],
    'price_doc': pred
})
prediction_df

,id,price_doc
0,30474,5.419743e+06
1,30475,8.015836e+06
2,30476,5.303301e+06
3,30477,6.122940e+06
4,30478,4.889536e+06
...,...,...
7657,38131,7.635401e+06
7658,38132,4.865493e+06
7659,38133,4.579440e+06
7660,38134,5.300390e+06


In [22]:
prediction_df.to_csv(output_path, index=False)

In [23]:
params

{'n_estimators': 957,
 'lambda_l2': 1.0179522290678555,
 'bagging_fraction': 0.803425892987424,
 'num_leaves': 730,
 'feature_fraction': 0.756874719068369,
 'max_depth': 9,
 'learning_rate': 0.010123433800404112,
 'subsample': 0.4105167842639978,
 'colsample_bytree': 0.5230497041584328,
 'min_data_in_leaf': 10,
 'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1}